# Convert DSSAT MINT Metadata to YAML

In [1]:
import mint_client
import requests
import json
import yaml

### Description

In [2]:
api_instance = mint_client.ModelApi(mint_client.ApiClient())
api_response = api_instance.get_models()

In [3]:
dssat_description = [i for i in api_response if i.id=='DSSAT'][0]

In [4]:
dssat_description

{'description': 'The Decision Support System for Agrotechnology Transfer '
                '(DSSAT) comprises dynamic crop growth simulation model for '
                'over 40 crops. The model simulates growth; development; and '
                'yield as a function of the soil-plant-atmosphere dynamics.',
 'has_documentation': ['https://dssat.net'],
 'has_model_category': ['Agriculture'],
 'has_software_version': [{'id': 'DSSAT_4.7',
                           'type': ['http://ontosoft.org/software#SoftwareVersion',
                                    'https://w3id.org/mint/modelCatalog#ModelVersion']}],
 'id': 'DSSAT',
 'label': 'DSSAT',
 'type': ['https://w3id.org/mint/modelCatalog#TheoryBasedModel',
          'https://w3id.org/mint/modelCatalog#Model']}

### Versions/Configs

In [5]:
api_instance = mint_client.ModelversionApi(mint_client.ApiClient())
version = api_instance.get_model_version('DSSAT')

In [6]:
version

{'id': 'DSSAT_4.7',
 'type': ['http://ontosoft.org/software#SoftwareVersion',
          'https://w3id.org/mint/modelCatalog#ModelVersion']}

In [7]:
api_instance = mint_client.ModelconfigurationApi(mint_client.ApiClient())
config_meta = api_instance.get_model_configuration(version.id)

In [8]:
config_meta

{'id': 'DSSAT_cfg_simple',
 'type': ['https://w3id.org/mint/modelCatalog#ModelConfiguration']}

#### Get configuration

In [9]:
api_instance = mint_client.ModelconfigurationApi(mint_client.ApiClient())
config = api_instance.get_model_configuration(config_meta.id)

### Get inputs and outputs

In [10]:
api_instance = mint_client.DatasetspecificationApi(mint_client.ApiClient())

In [11]:
datasets = api_instance.get_data_sets()

In [12]:
input_ids = [i.id for i in config.has_input]
output_ids = [i.id for i in config.has_output]

In [13]:
inputs = [i for i in datasets if i.id in input_ids]
outputs = [i for i in datasets if i.id in output_ids]

### Get variables

In [14]:
def get_variables(file):
    params = (
        ('io', f"https://w3id.org/mint/instance/{file.id}"),
        ('endpoint', 'https://endpoint.mint.isi.edu/ds/query'),
    )

    response = requests.get('https://query.mint.isi.edu/api/mintproject/MINT-ModelCatalogQueries/getI_OVariablesAndUnits', params=params)
    variables = response.json()['results']['bindings']

    variables_table = []
    for v in variables:
        e = {'name': v.get('label',{}).get('value'),
             'description': v.get('description',{}).get('value'),
             'metadata': {
                  'long_name': v.get('longName',{}).get('value'),
                  'units': v.get('unit',{}).get('value')
             },
             'standard_variables': [
                 {
                     'name': v.get('sn',{}).get('value'),
                     'ontology': 'SVO',
                     'uri': f"http://www.geoscienceontology.org/svo/svl/variable/1.0.0/#{v.get('sn',{}).get('value')}"
                 }
             ]
            }
             
        variables_table.append(e)
    return variables_table

In [15]:
def format_io_file(file):
    f = {}
    f['name'] = file.id
    f['type'] = file.has_format.replace("[","").replace("]", "").replace("'","")
    f['description'] = file.label
    f['variables'] = get_variables(file)
    return f

### Obtain Parameters

> **Note**: there are currently no parameters associated with the DSSAT configuration in MINT

In [16]:
api_instance = mint_client.ParameterApi(mint_client.ApiClient())

In [17]:
parameters = api_response = api_instance.get_parameters()

In [18]:
len(parameters)

49

### Generate Output Model

In [19]:
model = {}

In [20]:
model['name'] = dssat_description.id
model['description'] = dssat_description.description
model['maintainer'] = 'University of Florida'
model['url'] = dssat_description.has_documentation[0]

In [21]:
model['input_files'] = [format_io_file(i) for i in inputs]
model['output_files'] = [format_io_file(i) for i in outputs]

In [22]:
try:
    params = config.has_parameter
except:
    print("No parameters for configuration were found.")
    model['parameters'] = []

No parameters for configuration were found.


### Dump to YAML

In [23]:
ff = open('DSSAT-metadata.yaml', 'w+')
yaml.dump(model, ff, sort_keys=False, indent=2)